In [3]:
import math
import mpmath
import numpy as np
import qiskit as qc
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import IBMQ, execute, Aer
from qiskit.providers.aer import noise
import qiskit as qk
import time
import math
import sys

IBMQ.load_account()
IBMQ.get_provider().backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [4]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
	"""
	Call in a loop to create terminal progress bar
	@params:
		iteration   - Required  : current iteration (Int)
		total	   - Required  : total iterations (Int)
		prefix	  - Optional  : prefix string (Str)
		suffix	  - Optional  : suffix string (Str)
		decimals	- Optional  : positive number of decimals in percent complete (Int)
		length	  - Optional  : character length of bar (Int)
		fill		- Optional  : bar fill character (Str)
	"""
	percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
	filledLength = int(length * iteration // total)
	bar = fill * filledLength + '-' * (length - filledLength)
	print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
	# Print New Line on Complete
	if iteration == total: 
		print()

In [5]:
def mcx(qc, q, index, tgt):
	if len(index) == 0:
		qc.x(q[tgt])
	elif len(index) == 1:
		qc.cx(q[index[0]], q[tgt])
	elif len(index) == 2:
		qc.ccx(q[index[0]], q[index[1]], q[tgt])
	else:
		qc.barrier()
		qc.h(q[tgt])
		mcx(qc,q,index[:-1], tgt)
		qc.tdg(q[tgt])
		qc.cx(q[index[len(index)-1]], q[tgt])
		qc.t(q[tgt])
		mcx(qc,q,index[:-1], tgt)
		qc.tdg(q[tgt])
		qc.cx(q[index[len(index)-1]], q[tgt])
		qc.t(q[tgt])
		qc.h(q[tgt])
		qc.barrier()
		incrementer(qc, q, index)
		qc.barrier()
		j = 0
		for i in reversed(index[1:]):
			qc.u1(-math.pi/(pow(2, j+2)), q[i])
			j = j + 1
		qc.barrier()
		decrementer(qc, q, index)
		qc.barrier()
		j = 0
		for i in reversed(index[1:]):
			qc.u1(math.pi/(pow(2, j+2)), q[i])
			j = j + 1
		qc.u1(math.pi/(pow(2, len(index))), q[index[0]])
		qc.barrier()


def incrementer(qc, q, index):
	while len(index) > 2:
		mcx(qc, q, index[:-1], index[len(index)-1])
		index = index[:-1]
	if len(index) > 1:
		qc.cx(q[int(index[0])],q[int(index[1])])
	qc.x(q[int(index[0])])

def decrementer(qc, q, index):
	qc.x(q[int(index[0])])
	if len(index) > 1:
		qc.cx(q[int(index[0])],q[int(index[1])])
	i = 2
	while i < len(index):
		mcx(qc, q, index[:i], index[i])
		i = i + 1

def c_incrementer(qc, q, index, controls):
	for i in range(len(controls)):
		coerced = np.concatenate([index, controls[:len(controls)-i-1]])
		coerced = [int(i) for i in coerced]
		mcx(qc, q, coerced, int(controls[len(controls)-i-1]))

def c_decrementer(qc, q, index, controls):
	for i in range(len(controls)):
		coerced = np.concatenate([index, controls[:-len(controls)+i]])
		coerced = [int(i) for i in coerced]
		mcx(qc, q, coerced, int(controls[i]))

In [6]:
def mcu1(qc, q, index, tgt, theta):
        if len(index) == 0:
            qc.u1(theta, q[tgt])
        elif len(index) == 1:
            qc.cu1(theta, q[index[0]], q[tgt])
        else:
            qc.barrier()
            incrementer(qc, q, index)
            qc.barrier()
            j = 0
            for i in reversed(index[1:]):
                qc.u1(-theta/(pow(2, j+1)), q[i])
                j = j + 1
            qc.barrier()
            decrementer(qc, q, index)
            qc.barrier()
            j = 0
            for i in reversed(index[1:]):
                qc.u1(theta/(pow(2, j+1)), q[i])
                j = j + 1
            qc.u1(theta/(pow(2, len(index))), q[index[0]])

In [7]:
def mean_inversion_cnf(qc, q, n):
	## Hadamard gate
	for i in range(0, n):
		qc.h(q[i])
	## Pauli X gate
	for i in range(n):
		qc.x(q[i])
	mcx(qc, q, range(n), n)
	## Pauli X gate
	for i in range(n):
		qc.x(q[i])
	## Hadamard gate
	for i in range(n):
		qc.h(q[i])

In [8]:
def mean_inversion_fpaa(qc, q, n):
	qc.x(q[n+1])
	qc.h(q[n+1])
	## Hadamard gate
	for i in range(0, n):
		qc.h(q[i])
	## Pauli X gate
	for i in range(n):
		qc.x(q[i])
	mcx(qc, q, range(n), n+1)
	## Pauli X gate
	for i in range(n):
		qc.x(q[i])
	## Hadamard gate
	for i in range(n):
		qc.h(q[i])
	qc.h(q[n+1])
	qc.x(q[n+1])

In [9]:
def cnf_oracle(qc, q, n, a, oracle, index):
	b = math.floor(math.log(a,2))+1
	temp = [int(x) for x in np.binary_repr(a, width=b)]
	incr = []
	for i in range(b):
		incr.append(n+i+1)
	for j in range(a):
		qc.barrier()
		for i in range(n):
			if not abs(oracle[j][i]):
				qc.x(q[i])
		c_incrementer(qc, q, index[j],n+1+np.array(range(b)))
		for i in range(n):
			if not abs(oracle[j][i]):
				qc.x(q[i])
	qc.barrier()
	for i in range(b):
		qc.x(q[n+1+i])
	mcx(qc, q, incr, n)
	for i in range(b):
		qc.x(q[n+1+i])
	for j in range(a-1, -1, -1):
		qc.barrier()
		for i in range(n):
			if not abs(oracle[j][i]):
				qc.x(q[i])
		c_decrementer(qc, q, index[j],n+1+np.array(range(b)))
		for i in range(n):
			if not abs(oracle[j][i]):
				qc.x(q[i])

In [10]:
def fpaa_oracle(qc, q, n, clauses, oracle, index):
    theta = math.pi/clauses
    delta = pow(2,-0.5*pow(n,2))/2
    _lambda = pow(math.sin(theta),2)/4+pow(1/2-math.cos(theta)/2,2)
    L = int(math.ceil(2*math.log(2/delta)/math.sqrt(_lambda)))
    gamma = mpmath.cos(mpmath.acos(1/delta)/L)

    qc.barrier()
    A_matrix(qc, q, n, clauses, oracle, theta)
    qc.barrier()

    for k in range(1,L):
        alpha = abs(2*mpmath.acot(mpmath.tan(2*math.pi*(   k   )/L)*mpmath.sqrt(1-1/pow(gamma,-2))))
        beta = -abs(2*mpmath.acot(mpmath.tan(2*math.pi*(L-(k-1))/L)*mpmath.sqrt(1-1/pow(gamma,-2))))

        qc.h(q[n])
        U_matrix(qc, q, n, beta)
        qc.h(q[n])

        Adgr_matrix(qc, q, n, clauses, oracle, theta)

        U_matrix(qc, q, n, alpha)

        A_matrix(qc, q, n, clauses, oracle, theta)
        qc.barrier()
    qc.h(q[n])
    qc.barrier()

    qc.x(q[n])
    qc.x(q[n+1])
    qc.h(q[n+1])
    qc.cx(q[n],q[n+1])
    qc.h(q[n+1])
    qc.x(q[n+1])
    qc.x(q[n])


def U_matrix(qc, q, n, angle):
    qc.cx(q[n], q[n+1])
    qc.u1(float(angle), q[n+1])
    qc.cx(q[n], q[n+1])


def A_matrix(qc, q, n, clauses, oracle, theta):
    qc.h(q[n])
    for j in range(clauses):
        qc.barrier()
        for i in range(n):
            if not abs(oracle[j][i]):
                qc.x(q[i])
        mcu1(qc, q, range(n+1), n+1, theta)
        for i in range(n):
            if not abs(oracle[j][i]):
                qc.x(q[i])


def Adgr_matrix(qc, q, n, clauses, oracle, theta):
    for j in range(clauses):
        qc.barrier()
        for i in range(n):
            if not abs(oracle[j][i]):
                qc.x(q[i])
        mcu1(qc, q, range(n+1), n+1, -theta)
        for i in range(n):
            if not abs(oracle[j][i]):
                qc.x(q[i])
    qc.barrier()
    qc.h(q[n])

In [11]:
def grover_search_cnf(n, clauses, shots, oracle, index, nr_solutions):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = math.floor((math.pi*math.sqrt(pow(2, n)/nr_solutions))/4)
    a = math.floor(math.log(clauses,2)+1)

    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if (n+1) + a > 24:
        print("Too much qubits needed! (", (n+1) + a,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister( n +1 + a )
    c = ClassicalRegister( n + 1 + a )
    qc = QuantumCircuit(q, c)
    
    for i in range(n):
        qc.h(q[i])
    qc.x(q[n])
    qc.h(q[n])

    for i in range(iterations):
        cnf_oracle(qc, q, n, clauses, oracle, index)
        mean_inversion_cnf(qc, q, n)

        # for i in range(n,n+a):
        #     qc.measure(q[i], c[0])
        #     qc.x(q[i]).c_if(c,1)

    for i in range(n):
        qc.measure(q[i], c[i])

    backend = qk.BasicAer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots, max_credits=3)
    print(job.result().get_counts())
    # Draw the circuit
    print(qc)

In [12]:
def grover_search_fpaa(n, shots, oracle, index, nr_solutions):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = math.floor((math.pi*math.sqrt(pow(2, n)/nr_solutions))/4)
    
    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if n+2 > 24:
        print("Too much qubits needed! (", n+2,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister( n + 2 )
    c = ClassicalRegister( n + 2 )
    qc = QuantumCircuit(q, c)

    for i in range(math.floor(n/2)):
        qc.swap(q[i],q[n-i-1])
        
    for i in range(n):
        qc.h(q[i])
        
    for i in range(iterations):
        fpaa_oracle(qc, q, n, clauses, oracle, index)
        mean_inversion_fpaa(qc, q, n)

        # for i in range(n,n+a):
        #     qc.measure(q[i], c[0])
        #     qc.x(q[i]).c_if(c,1)

    for i in range(math.floor(n/2)):
        qc.swap(q[i],q[n-i-1])
        
    for i in range(n):
        qc.measure(q[i], c[i])

    backend = qk.BasicAer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=shots, max_credits=3)
    print(job.result().get_counts())
    # Draw the circuit
    print(qc)

In [15]:
def make_cnf():
    # 1 = variable, 0 = negated variable, -1 = not in clause
    cnf = [
        [0, 0, -1],
        [0, 1, -1],
        [1, 0, -1],
        [1, -1, 0]
    ]
    # values are the indexes of each variable in the clause
    index = [
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 2]
    ]
    # number of variables
    n = 3
    # number of clauses
    clauses = 4
    # number of solutions
    nr_solutions = 1
    return n, clauses, cnf, index, nr_solutions

In [16]:
n, clauses, oracle, index, nr_solutions = make_cnf()
shots = 100

grover_search_cnf(n, clauses, shots, oracle, index, nr_solutions)
# grover_search_fpaa(n, shots, oracle, index, nr_solutions)

{'0000010': 2, '0000000': 1, '0000111': 97}
         ┌───┐      ░ ┌───┐ ░       ░                                        »
q1_0: |0>┤ H ├──────░─┤ X ├─░───────░────────■─────────────────────■─────────»
         ├───┤      ░ ├───┤ ░       ░        │                     │         »
q1_1: |0>┤ H ├──────░─┤ X ├─░───────░────────■─────────────────────■─────────»
         ├───┤      ░ └───┘ ░       ░        │                     │         »
q1_2: |0>┤ H ├──────░───────░───────░────────┼─────────────────────┼─────────»
         ├───┤┌───┐ ░       ░       ░        │                     │         »
q1_3: |0>┤ X ├┤ H ├─░───────░───────░────────┼─────────────────────┼─────────»
         └───┘└───┘ ░       ░       ░        │                     │         »
q1_4: |0>───────────░───────░───────░────────┼───────────■─────────┼─────────»
                    ░       ░       ░        │           │         │         »
q1_5: |0>───────────░───────░───────░────────┼───────────┼─────────┼─────────»
        

In [83]:
def grover_search_cnf_noisy(n, clauses, shots, oracle, index, nr_solutions, value):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = math.floor((math.pi*math.sqrt(pow(2, n)/nr_solutions))/4)
    a = math.floor(math.log(clauses,2)+1)

    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if (n+1) + a > 24:
        print("Too much qubits needed! (", (n+1) + a,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister( n +1 + a )
    c = ClassicalRegister( n + 1 + a )
    qc = QuantumCircuit(q, c)
    
    for i in range(n):
        qc.h(q[i])
    qc.x(q[n])
    qc.h(q[n])

    for i in range(iterations):
        cnf_oracle(qc, q, n, clauses, oracle, index)
        mean_inversion_cnf(qc, q, n)

        for i in range(n,n+a):
            qc.measure(q[i], c[0])
            qc.x(q[i]).c_if(c,1)

    for i in range(n):
        qc.measure(q[i], c[i])


    backend = Aer.get_backend('qasm_simulator')

    # Execute noisy simulation and get counts
    device = IBMQ.get_provider().get_backend('ibmq_london')
    properties = device.properties()
    coupling_map = device.configuration().coupling_map

    for _, qubit_props in enumerate(properties.qubits):
        for item in qubit_props:
            if item.name == 'T1':
                item.value = value
            if item.name == 'T2':
                item.value = value
    for gate in properties.gates:
        for item in gate.parameters:
            if item.name == 'gate_error':
                item.value = 0.00000001
    gate_times = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200), ('cx', None, 0)
    ]
    # Construct the noise model from backend properties
    # and custom gate times
    noise_model = noise.device.basic_device_noise_model(properties,readout_error=False, thermal_relaxation=True, gate_times=gate_times)

    # Get the basis gates for the noise model
    basis_gates = noise_model.basis_gates

    result = execute(qc, backend=backend, shots=100, noise_model=noise_model, basis_gates=basis_gates).result()
    counts = result.get_counts(qc)
    return counts

In [80]:
def grover_search_fpaa_noisy(n, shots, oracle, index, nr_solutions, value):
    if pow(2, n)/nr_solutions < 4:
        print("this does not work with grover.")
        print("total/solutions need to be larger than 4.")
        print("brute force will solve it efficiently.")
        return
    iterations = math.floor((math.pi*math.sqrt(pow(2, n)/nr_solutions))/4)
    
    ## Too much qubits needed for operation (max qubits for simulator is 24)
    if n+2 > 24:
        print("Too much qubits needed! (", n+2,")")
        print("Max qubits 24")
        return

    ## circuit generation
    q = QuantumRegister( n + 2 )
    c = ClassicalRegister( n + 2 )
    qc = QuantumCircuit(q, c)

    for i in range(math.floor(n/2)):
        qc.swap(q[i],q[n-i-1])
        
    for i in range(n):
        qc.h(q[i])
        
    for i in range(iterations):
        fpaa_oracle(qc, q, n, clauses, oracle, index)
        mean_inversion_fpaa(qc, q, n)

        for i in range(n,n+2):
            qc.measure(q[i], c[0])
            qc.x(q[i]).c_if(c,1)

    for i in range(math.floor(n/2)):
        qc.swap(q[i],q[n-i-1])
        
    for i in range(n):
        qc.measure(q[i], c[i])

    backend = Aer.get_backend('qasm_simulator')

    # Execute noisy simulation and get counts
    device = IBMQ.get_provider().get_backend('ibmq_london')
    properties = device.properties()
    coupling_map = device.configuration().coupling_map

    for _, qubit_props in enumerate(properties.qubits):
        for item in qubit_props:
            if item.name == 'T1':
                item.value = value
            if item.name == 'T2':
                item.value = value
    for gate in properties.gates:
        for item in gate.parameters:
            if item.name == 'gate_error':
                item.value = 0.00000001
    gate_times = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200), ('cx', None, 0)
    ]
    # Construct the noise model from backend properties
    # and custom gate times
    noise_model = noise.device.basic_device_noise_model(properties,readout_error=False, thermal_relaxation=True, gate_times=gate_times)

    # Get the basis gates for the noise model
    basis_gates = noise_model.basis_gates

    result = execute(qc, backend=backend, shots=100, noise_model=noise_model, basis_gates=basis_gates).result()
    counts = result.get_counts(qc)
    return counts

In [84]:
start_val = 1
end_val   = 100
runs = 10
shots = 100
n, clauses, oracle, index, nr_solutions = make_cnf()

dist = end_val-start_val+(end_val - start_val) / (runs-1)
values = np.arange(start_val,end_val+(end_val - start_val) / (runs-1), dist/runs)
data = []

printProgressBar(0, len(values), prefix = 'Progress:', suffix = 'Complete', length = len(values))
for i in range(len(values)):
    data.append(grover_search_cnf_noisy(n, clauses, shots, oracle, index, nr_solutions, values[i]))
#     data.append(grover_search_fpaa_noisy(n, shots, oracle, index, nr_solutions, values[i]))
    printProgressBar(i, len(values), prefix = 'Progress:', suffix = 'Complete', length = len(values))

print(data)

[{'00000': 30, '00010': 26, '00001': 30, '00011': 14}, {'00000': 10, '00010': 12, '00001': 20, '00011': 58}, {'00000': 9, '00010': 4, '00001': 9, '00011': 78}, {'00000': 3, '00010': 6, '00001': 9, '00011': 82}, {'00000': 1, '00010': 3, '00001': 8, '00011': 88}, {'00000': 4, '00010': 2, '00001': 4, '00011': 90}, {'00000': 1, '00010': 4, '00001': 3, '00011': 92}, {'00000': 3, '00010': 3, '00001': 6, '00011': 88}, {'00000': 2, '00010': 3, '00001': 6, '00011': 89}, {'00000': 2, '00010': 1, '00001': 1, '00011': 96}]
